# Описание и пример простейшей реализации алгоритма линейной регрессии

Регрессия (приближение к среднему) - это способ выбрать из семейства функций ту, которая минимизирует функцию потерь. Функция потерь или лосс функцция (калька с английского) характеризует насколько пробная функция отклоняется от значений в заданных точках.
Линейную регрессия называют линейной не из-за вида базисной функции (первая степень), а из-за того, что функции из которых мы выбираем, представляют собой линейную комбинацию (сумму с коэффициентами) наперед заданных базисных функций $f{i}$. Например, x1 + x2^2. Сумма базисных функций х1 и x2^2 - линейная комбинация.

f = $\sum_{i}w{i}f{i}$



### Функция потерь
Для промежуточной оценки модели в процессе работы алгоритма берут сумму расстояний от заданных точек до кривой (прямой) пробной функции или зависящие от этих расстояний величины. Сумма модулей отклонений $\sum_i |f(x_i) - y_i|$ или Least Absolute Distance (LAD) регрессия.
Одной из популярных лосс функциий стала сумма квадаратов отклонений регрессанта от модели или Sum of Squared Errors (SSE). Производная квадратичной лосс функции линейная и непрерывная, поэтому легко ищется. Минус суммы квадратов отклонений - при увеличении количества аргументов (фичей) Сумма квадратов отклонений очень быстро возрастает. Поэтому её делят на количество фичей и получают среднеквадратичную ошибку _MSE_:

MSE = $\dfrac{1}{N} \sum_{i=1}^{N}(y_{act} - y_{pred})^2 = \dfrac{1}{N} \sum_{i=1}^{N}(y_{act} - (\omega_0 + x_{i1} \cdot \omega_1 + x_{i2} \cdot \omega_2 + ... + x_{in} \cdot \omega_n))^2$


### Простейшая реализация класса линейной регрессии

Ниже реализован простейший класс Линейной регрессии.  

на _train_ выборке алгоритму необходиму обучиться, на _test_ выборке проверить свой результат. Метрика для проверки результата для линейной регрессии - _MSE_, . Метрики реализовать внутри класса.

В качестве функции потерь, необходимо выбрать _MSE_ - для линейной регрессии. 

После можно сравнить свой результат со стандартной Линейной регрессией, реализованной в _sklearn_

In [2]:
import pandas as pd
import numpy as np

In [3]:
train_0 = pd.read_csv('aux/LinRegtrain.csv').drop('Unnamed: 0', axis=1)
test_0 = pd.read_csv('aux/LinRegtest.csv').drop('Unnamed: 0', axis=1)
test_0

,x1,x2,x3,y
0,27.706438,23.195254,16.552800,16.731168
1,11.606632,19.186824,1.819811,40.515215
2,12.096257,13.350714,10.882195,-11.999238
3,14.172638,19.603994,7.730774,9.645480
4,15.984409,9.527360,3.449821,22.803054
...,...,...,...,...
195,14.356981,19.287363,7.788886,35.289736
196,25.727621,28.236211,9.571604,57.570480
197,13.434383,24.280476,4.162805,40.580514
198,13.973079,6.634176,1.918399,20.631645


In [8]:
#Нормализуем обучающие данные train - приводим их к такому виду, что среднее арифметическое каждого столбца становится равно 0, а дисперсия - 1. Из каждого значения x1 вычитается одно и то же значение и полученное значение делится на дисперсию.
# Алгоритм покажет нужные значения таргета за счет изменения свободного члена, а MSE значительно уменьшится.
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
train = pd.DataFrame()
test = pd.DataFrame()
train[['x1', 'x2', 'x3']] = ss.fit_transform(train_0[['x1', 'x2', 'x3']] )
train[['y']] = train_0[['y']]

test[['x1', 'x2', 'x3']] = ss.transform(test_0[['x1', 'x2', 'x3']] ) # С теми же значениями трансформируем тестовую выборку
test[['y']] = test_0[['y']]
print(sum(train['x1']))
test[['x1', 'x2', 'x3']]


1.865174681370263e-14


,x1,x2,x3
0,1.311696,0.674212,2.486236
1,-1.529648,-0.107172,-0.632298
2,-1.443237,-1.244834,1.285938
3,-1.076791,-0.025851,0.618876
4,-0.757044,-1.990140,-0.287274
...,...,...,...
195,-1.044257,-0.087573,0.631177
196,0.962468,1.656871,1.008525
197,-1.207080,0.885759,-0.136356
198,-1.112010,-2.554123,-0.611430


In [9]:
class LinReg:
    def __init__(self, r_coef): #  Занесим все необходимые вещи в атрибуты класса(коэффициенты регуляризации, вид, инициализацию весов)
        self.r_coef = r_coef  #коэффицент регуляризации
        self.w_0 = 0
        self.w_1 = 1
        self.w_2 = 1
        self.w_3 = 1

#градиент (линия градиента) равна векторам частных производных: gradient1 = -2 * (X.iloc[:, 0] * error).mean()


    def fit(self, X, y): # Реализация градиентного спуска, чтобы найти самые лучшие веса
        for i in range(1000):
            yhat = X.iloc[:, 0] * self.w_1 + X.iloc[:, 1] * self.w_2 + X.iloc[:, 2] * self.w_3 + self.w_0
            error = y.to_numpy() - yhat
            gradient1 = -2 * (X.iloc[:, 0] * error).mean()
            gradient2 = -2 * (X.iloc[:, 1] * error).mean()
            gradient3 = -2 * (X.iloc[:, 2] * error).mean()
            gradient0 = -2 * error.mean()
            self.w_1 = self.w_1 - self.r_coef * gradient1
            self.w_2 = self.w_2 - self.r_coef * gradient2
            self.w_3 = self.w_3 - self.r_coef * gradient3
            self.w_0 = self.w_0 - self.r_coef * gradient0

    def predict(self, X):
        return X.iloc[:, 0] * self.w_1 + X.iloc[:, 1] * self.w_2 + X.iloc[:, 2] * self.w_3 + self.w_0


In [10]:
lreg = LinReg(0.01)

lreg.fit(train.iloc[:, 0:3], train.iloc[:, 3])

lreg.w_1, lreg.w_2, lreg.w_3, lreg.w_0

(11.667360670089087, 5.318342681639542, -14.08200198188564, 45.93671038062178)

In [11]:

c = lreg.predict(test[['x1', 'x2', 'x3']])
c

0      29.815258
1      36.423810
2       4.368910
3      24.520905
4      30.565150
         ...    
195    24.399007
196    51.775935
197    38.484213
198    27.988951
199    31.986937
Length: 200, dtype: float64

In [12]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

X, y = train[['x1','x2','x3']], train['y']
lr.fit(X, y)

lr.coef_, lr.intercept_

(array([ 11.66736071,   5.31834268, -14.08200203]), 45.93671045793176)